In [8]:
!pip install streamlit

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/10.1 MB ? eta -:--:--
   ----------------------- ---------------- 6.0/10.1 MB 33.0 MB/s eta 0:00:01
   ---------------------------------------- 10.1/10.1 MB 33.8 MB/s  0:00:00
   ---------------------------------------- 0.0/731.2 kB ? eta -:--:--
   ---------------------------------------- 731.2/731.2 kB 34.7 MB/s  0:00:00
   ---------------------------------------- 0.0/7.0 MB ? eta -:--:--
   ---------------------------------------- 7.0/7.0 MB 39.0 MB/s  0:00:00
   ---------------------------------------- 0.0/6.9 MB ? eta -:--:--
   ---------------------------------------- 6.9/6.9 MB 42.0 MB/s  0:00:00
   ---------------------------------------- 0.0/26.1 MB ? eta -:--:--
   ------------- -------------------------- 8.9/26.1 MB 44.0 MB/s eta 0:00:01
   -------------------------- ------------- 17.0/26.1 MB 43.2 MB/s eta 0:00:01
   ---------------------------------

In [11]:
import pandas as pd
import re
import streamlit as st

# Subindo e tratando os dados
exportacao = pd.read_csv('ExpVinho.csv', sep=';')
exportacao_visualizador = pd.DataFrame(exportacao)

# excluindo a coluna 'Id', porque não é uma informação útil para a análise
exportacao = exportacao.drop('Id', axis=1)

# o problema de negócio pede que sejam analisados os últimos 15 anos
# como cada coluna representa um valor de quantidade e de valor monetário por ano, será conservado somente as últimas 30 colunas
colunas = exportacao.columns[-30:]
colunas = colunas.insert(0, 'País')
exportacao = exportacao[colunas]

# corrigindo a atribuição
exportacao.loc[2, 'País'] = 'Alemanha'

# retirando do dataframe os países que não compraram vinho com o Brasil
exportacao['total'] = exportacao.sum(numeric_only=True, axis=1)
paises_comercio_zero = exportacao[exportacao['total'] == 0]
exportacao = exportacao.drop(paises_comercio_zero.index, axis=0)
exportacao = exportacao.drop('total', axis=1)

# exibindo no Streamlit (novo formato)
st.dataframe(exportacao, width='stretch')


2025-10-19 09:00:59.422 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-19 09:00:59.423 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-19 09:00:59.423 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [ ]:
# Separando os valores das variáveis 'quantidade' e 'valor' que hoje estão representadas no dataframe como ANO e ANO.1 respectivamente
quantidadel = []
valoruss = []
for item in exportacao.columns:
    if item == 'País':
        quantidadel.append(item)
        valoruss.append(item)
    elif len(item) == 4:
        quantidadel.append(item)
    else:
        valoruss.append(item)
# criando dataframes separados para quantidade e valor
quantidade_l = exportacao[quantidadel]
quantidade_l = quantidade_l.set_index('País')
valor_uss = exportacao[valoruss]
valor_uss = valor_uss.set_index('País')
    

In [ ]:
# retirando o '.1' do valor da coluna para que futuramente seja possível juntar as tabelas
colunas = valor_uss.columns.str[:-2]
valor_uss.columns = colunas
valor_uss.head(2)

In [ ]:
# fazendo o melt e criando um dataframe único com informações de quantidade e valor
valor_uss2 = valor_uss.reset_index('País')
quantidade_l2 = quantidade_l.reset_index('País')

quantidade_l2 = quantidade_l2.melt(id_vars=['País'], value_vars=quantidade_l.columns)
valor_uss2 = valor_uss2.melt(id_vars=['País'], value_vars=valor_uss.columns)

# renomeando as colunas para a correta identificação das variáveis
quantidade_l2.columns = ['pais_destino', 'ano', 'quantidade_l']
valor_uss2.columns = ['pais_destino', 'ano', 'valor_uss']

In [ ]:
# fazendo o .join() entre as duas tabelas para tranasformá-la em uma só
exportacao_long = quantidade_l2.join(valor_uss2['valor_uss'])
exportacao_long.head(2)


In [ ]:
exportacao_long_por_pais = exportacao_long.groupby(by='pais_destino').sum(numeric_only=True)
exportacao_long_por_pais = exportacao_long_por_pais.sort_values(by='valor_uss', ascending=False)
exportacao_long_por_pais.sum()


In [ ]:
exportacao_long_por_pais_top10 = exportacao_long_por_pais.head(10)
exportacao_long_por_pais_top10 = exportacao_long_por_pais_top10.reset_index('pais_destino')
exportacao_long_por_pais_top10.head()

In [ ]:
df_aux = pd.read_excel('codigo_iso-alpha_top10_paises_importadores_vinho.xlsx')
df_aux.head

In [ ]:
# fazendo o .join() entre as duas tabelas para tranasformá-la em uma só
exportacao_long_por_pais_top10 = exportacao_long_por_pais_top10.join(df_aux[['cod_num', 'iso_alpha']])
exportacao_long_por_pais_top10.head()


In [ ]:
exportacao_long_por_pais_top10['pais_origem'] = 'Brasil'
exportacao_long_por_pais_top10.head()

In [ ]:
exportacao_long_por_pais['dolar_por_litro'] = exportacao_long_por_pais['valor_uss'] / exportacao_long_por_pais['quantidade_l']
exportacao_long_por_pais.head()

In [ ]:
# incluindo a coluna total nos dataframe
quantidade_l["Total_quantidade"] = quantidade_l.sum(axis=1)
valor_uss["Total_valor"] = valor_uss.sum(axis=1)

In [ ]:
# criando dataframe com paises que não compraram vinho do brasil
paises_comercio_zero = valor_uss[valor_uss['Total_valor'] == 0]
paises_comercio_zero

In [ ]:
# retirando dos dataframe quantidade_l e valor_uss os países que não compraram vinho com o brasil
quantidade_l_s = quantidade_l.drop(paises_comercio_zero.index, axis=0)
valor_uss_s = valor_uss.drop(paises_comercio_zero.index, axis=0)

# ordenando por maior quantidade e maior valor
quantidade_l_ordenado_por_total = quantidade_l_s.sort_values(by='Total_quantidade', ascending = False)
valor_uss_s_ordenado_por_total = valor_uss_s.sort_values(by='Total_valor', ascending = False)

# passo 1: retirando a coluna total dos dataframes
quantidade_l_ordenado_por_total = quantidade_l_ordenado_por_total.drop('Total_quantidade', axis=1)
valor_uss_s_ordenado_por_total = valor_uss_s_ordenado_por_total.drop('Total_valor', axis=1)

# passo 2: criando um dataframe com 10 países que mais compraram vinho do brasil
quantidade_l_ordenado_por_total_top10 = quantidade_l_ordenado_por_total.head(10)
valor_uss_s_ordenado_por_total_top10 = valor_uss_s_ordenado_por_total.head(10)

# passo 3: transpondo os dataframe para que seja possível visualizar a serie temporal
quantidade_l_ordenado_por_total_top10 = quantidade_l_ordenado_por_total_top10.T
valor_uss_s_ordenado_por_total_top10 = valor_uss_s_ordenado_por_total_top10.T

# passo 4: mostrando os dataframes
display(quantidade_l_ordenado_por_total_top10.head(2))
display(valor_uss_s_ordenado_por_total_top10.head(2))